# Introduction

Few month ago a youtuber presented a small bot on twitter which tweet everyday, the sum of delayed train in France. This gaves me the idea of wishing a happy birthday to celebrities with a specific account automatically. In this notebook, we will extract birthday and twitter account from a website using web scrapping and store data in a small database to deplay the bot afterward.

In [1]:
import sqlite3
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
pattern = re.compile("[0-9]+")

# DB creation

First let's create the SQLite db. It's small, easy to deploy without any installation so perfect for this. We will need 1 table to store :

- name
- birthday
- twitter account
- url on the website we scrap (optionnal but I'll use it for updates)
- country (for Now I'll send tweet only to French celebrities
- if the person is dead or not

In [ ]:
# conn = sqlite3.connect('F:/data/birthday.db')
# c = conn.cursor()
# c.execute('''DROP TABLE IF EXISTS birthday''')
# c.execute('''CREATE TABLE birthday
#              (birthday date_text, name text, twitter text, url text, country text, dead integer)''')
# # c.execute("INSERT INTO birthday VALUES ('1990-05-15','Nicolas M.','@Coni631', '', 'France', '0')")
# conn.commit()
# conn.close()

We can now check if it works.

In [ ]:
# conn = sqlite3.connect('F:/data/birthday.db')
# c = conn.cursor()
# c.execute('SELECT * FROM birthday')
# print(c.fetchone())
# conn.close()

And it's fine

# Scapping

For the scrapping, we will use https://anniversaire-celebrite.com/ as a website to scrap the info. First of all, we will go throught every day of the year to gather all persons having a birthday at this date. For each persons, we will open their own page to scrap the account if we have it.To do so let's create few fonctions :

In [2]:
def check_if_exist(name):
    conn = sqlite3.connect('F:/data/birthday.db')
    c = conn.cursor()
    c.execute("SELECT * FROM birthday WHERE name=?", (name,))
    if c.fetchone() is None:
        res = False
    else:
        res = True
    conn.close()
    return res
    
def get_Twitter(url):
    actor_page = requests.get(url)
    DOM = BeautifulSoup(actor_page.text, "html.parser")
    counter = DOM.find("div", attrs={"class" : "twitter"})
    if counter is not None:
        url = counter.find("a")["href"]
        account = "@" + url.split("/")[-1]
        return account
    return ""

def add_data(birthday, name, twitter, url, country, dead):
    conn = sqlite3.connect('F:/data/birthday.db')
    c = conn.cursor()
    q = "INSERT INTO birthday VALUES ('{}','{}','{}', '{}', '{}', '{}')".format(birthday, name, twitter, url, country, dead)
    c.execute(q)
    conn.commit()
    conn.close()

def page_to_actor(DOM, month, day):
    birthday_block = DOM.find_all("div", attrs={"class":"columns"})[3]
    actors = birthday_block.find_all("div", attrs={"class":"column col-2 col-xl-2 col-lg-3 col-md-4 col-xs-6"})
    for actor in actors:
        data = actor_to_data(actor, month, day)
    
def actor_to_data(actor, month, day):
    result = {}
    alts = actor.find_all("img")
    year_txt = actor.find("div", {"class" : "celannee tiny"}).getText()
    year = int(pattern.search(year_txt).group(0))
    result["name"] = re.sub("'","''", alts[0]['alt'])
    result["country"] = re.sub("'","''", alts[1]['alt'])
    result["birthday"] = "{}-{:02d}-{:02d}".format(year, month, day)
    result["dead"] = len(actor.find("div", {"class" : "celage"}).find_all("strong")) - 1
    result["url"] = "https://anniversaire-celebrite.com/" + actor.find("a")["href"]
    
    alread_in = check_if_exist(result["name"])
    if not alread_in:
        if result["dead"] == 0:
            result["twitter"] = get_Twitter(result["url"])
        else:
            result["twitter"] = ""
        add_data(**result)
    else:
        print("Skip " + result["name"])

Now we just have to run it for all date of the year

In [ ]:
done = True
for month in range(1, 13):
    for day in range(1, 32):
        if month == 1 and day == 10:
            done = False
            
        if not done:
            print("extracting {:02d}/{:02d}".format(day, month))
            url = "https://anniversaire-celebrite.com/ok,annee,{:02d},{:02d}.html".format(month, day)
            page = requests.get(url)
            soup = BeautifulSoup(page.text, "html.parser")
            page_to_actor(soup, month, day)

# Add new persons

Sometimes, the website add celebrities to their website, we can create a small function to check it and store new ones.

In [3]:
def getDataFromProfil(url):
    actor_page = requests.get(url)
    DOM = BeautifulSoup(actor_page.text, "html.parser")
    birthday = DOM.find(itemprop="birthDate")["datetime"]
    dead = 1 if DOM.find(itemprop="deathDate") else 0
    counter = DOM.find("div", attrs={"class" : "twitter"})
    if counter is not None:
        url = counter.find("a")["href"]
        account = "@" + url.split("/")[-1]
        return birthday, account, dead
    return birthday, "", dead

In [7]:
url = "https://anniversaire-celebrite.com/last.html?tri=4" #[1 = homme, 2 femme, 3 mort, 4 vivant]
page = requests.get(url)
DOM = BeautifulSoup(page.text, "html.parser")
birthday_block = DOM.find_all("div", attrs={"class":"columns"})[3]
actors = birthday_block.find_all("div", attrs={"class":"column col-2 col-xl-2 col-lg-3 col-md-4 col-xs-6"})
for actor in actors:
    result = {}
    alts = actor.find_all("img")
    year_txt = actor.find("div", {"class" : "celannee tiny"}).getText()
    year = int(pattern.search(year_txt).group(0))
    result["name"] = re.sub("'","''", alts[0]['alt'])
    result["country"] = re.sub("'","''", alts[1]['alt'])
    result["url"] = "https://anniversaire-celebrite.com/" + actor.find("a")["href"]
    
    alread_in = check_if_exist(result["name"])
    if not alread_in:
        birthday, twitter, dead = getDataFromProfil(result["url"])
        result["twitter"] = twitter
        result["birthday"] = birthday
        result["dead"] = dead
        print("Add " + result["name"])
        add_data(**result)
    else:
        print("Skip " + result["name"])

Skip Yves Pignot
Skip Robbie Coltrane
Skip  Sananas
Skip Benjamin Pavard
Skip Vicky Leandros
Skip Mary Steenburgen
Skip Markus Persson
Skip Catherine Alric
Skip Lizzie Brocheré
Skip Peter Bogdanovich
Add Ryan O''Neal
Skip  Lord Vinheteiro
Skip  Jiminy Cricket
Skip  Fée Clochette
Skip John Boyega
Skip James Dyson
Skip Franck Provost
Skip Jane March
Skip Luc Thuillier
Skip  Le Petit Chaperon rouge
Skip  Blanche-Neige
Skip Roger Rabbit
Skip Sam le pirate
Skip  Speedy Gonzales
Skip Alice Belaïdi
Add  E.T., l''extra-terrestre
Skip Angelo Branduardi
Skip François-Éric Gendron
Skip  Valli
Skip John Hannah
Skip Benoît Brisefer
Skip Ritchie Blackmore
Skip Roger Glover
Skip Derren Brown
Skip  Yoda
Skip  Heidi
Skip Harold Faltermeyer
Skip Paul Le Guen
Skip Pascale Petit
Skip Sandie Shaw
Skip Bernard Minet
Add  L''Élève Ducobu
Skip  Bill
Skip  Boule
Skip Mohamed Salah
Skip Luis Suárez
Skip Yoann Sover
Skip Caroline Munoz
Skip Elle Fanning
Skip Greta Thunberg
Skip Jeri Ryan
Skip Christopher Atkins


# Check Profil for update

Another thing to consider is the case where there is changes. For example, we have several celebrities without twitter account. We could also create a function to check all celebrities without official twitter regularly (that's why I saved the url) and add the account if it's added. Some people may also die and we should not wish them a Happy Birthday :(

In [9]:
conn = sqlite3.connect('F:/data/birthday.db')
c = conn.cursor()
c.execute("SELECT name, url FROM birthday WHERE twitter = '' AND dead == 0")
l = c.fetchall()
conn.close()

In [12]:
# skip = True
for name, url in l:
#     if name == "Paul-henri Mathieu":
#         skip = False
#     if not skip:
    print("Checking", name)
    birthday, twitter, dead = getDataFromProfil(url)
    if twitter != "" or dead == 1:
        print("Update on", name)
        conn = sqlite3.connect('F:/data/birthday.db')
        c = conn.cursor()
        c.execute("UPDATE birthday SET twitter = ?, dead = ? WHERE name = ?", (twitter, dead, name))
        conn.commit()
        conn.close()

Checking Paul-henri Mathieu
Update on Paul-henri Mathieu
Checking Olivier Martinez
Checking John Lasseter
Checking Charlotte Julian
Checking Jean-Louis Bianco
Checking  Jordy
Checking Christine Lemler
Checking Emily Watson
Checking Steven Soderbergh
Checking Stéphane Thebaut
Checking Dominique Rocheteau
Checking Jean-François Mattéi
Checking Faye Dunaway
Checking Kellita Smith
Checking Frédéric Lordon
Checking  Yellowman
Checking Catherine Trautmann
Checking Marius Trésor
Checking Richard Bohringer
Checking Jean-Claude Narcy
Checking Naveen Andrews
Checking Mathilde Seigner
Checking Mick Taylor
Checking Françoise Hardy
Checking Alison Arngrim
Checking Philippe Starck
Checking Franz-Olivier Giesbert
Checking Stefan Edberg
Checking Tippi Hedren
Checking Skeet Ulrich
Checking James Denton
Checking Gérard Hernandez
Checking Buzz Aldrin
Checking Vanessa Hessler
Checking Nicolas Mahut
Checking Philippe Lelièvre
Checking Geena Davis
Checking  Bigflo
Checking Raica Oliveira
Checking Frank Lebo

Checking Yves Boisset
Checking Michael Caine
Checking Philippe Alexandre
Checking Terence Trent D'Arby
Checking Julie Bataille
Checking Joaquim de Almeida
Checking Jacques Doillon
Checking David Cronenberg
Checking Anne Charrier
Checking Cristiana Reali
Checking Isabelle Huppert
Checking Erik Estrada
Checking Victor Garber
Checking Jean-Claude Dauphin
Checking Guesch Patti
Checking Marc Meneau
Checking Kurt Russell
Checking Daniel Lavoie
Checking Anne Girouard
Checking Marine Delterme
Checking Vanessa Lynn Williams
Checking Arnaud Lagardère
Checking Xavier Deluc
Checking Jacques Secrétin
Checking Patrick Chesnais
Checking Arlette Laguiller
Checking Ivan Levaï
Checking Kim Rae-won
Checking Rachel Blanchard
Checking Mallaury Nataf
Checking Bruce Willis
Checking Harvey Weinstein
Checking Ursula Andress
Checking  Ruby Rose
Checking Anouk Grinberg
Checking Holly Hunter
Checking Spike Lee
Checking Élizabeth Bourgine
Checking William Hurt
Checking Danièle Gilbert
Checking Marthe Villalonga
Ch

Checking Melissa Gilbert
Checking Marie Myriam
Checking David Keith
Checking Isabelle Alonso
Checking Catherine Laborde
Checking Katia Tchenko
Checking Keith Jarrett
Checking Dave Gahan
Checking John Corbett
Checking Béatrice Schönberg
Checking Candice Bergen
Checking Ion Țiriac
Checking Sylvain Wiltord
Checking Élodie Hesme
Checking Linda Evangelista
Checking Emmanuelle Devos
Checking Bruno Wolkowitch
Checking  Bono
Checking Bruno Madinier
Checking Laurent Spielvogel
Checking Marina Vlady
Checking Jean Becker
Checking Françoise Fabian
Checking Armel Le Cléac'h
Checking Annelise Hesme
Checking Benoît Magimel
Checking Thomas Hugues
Checking Emmanuelle Haïm
Checking Isabelle Mergault
Checking Jean-François Dérec
Checking Fanny Cottençon
Checking Corinne Lepage
Checking  Ringo
Checking Jean Sarrus
Checking Domhnall Gleeson
Checking Malin Åkerman
Checking Sami Bouajila
Checking Stephen Baldwin
Checking Emilio Estevez
Checking Ving Rhames
Checking Kim Greist
Checking Olivier Lejeune
Checkin

Checking Florence Thomassin
Checking Nancy Allen
Checking Jeff Beck
Checking Brigitte Fontaine
Checking Cécile Cassel
Checking Vladimir Kramnik
Checking Pierre-François Martin-Laval
Checking Erica Gimpel
Checking Charlotte Kady
Checking Christine Albanel
Checking Robert Charlebois
Checking Samuel Benchetrit
Checking Dany Boon
Checking Greg LeMond
Checking Véronique Genest
Checking Rachid Arhab
Checking Maxime Bossis
Checking Gilberto Gil
Checking Yassine Belattar
Checking Tobey Maguire
Checking  J. J. Abrams
Checking Franck Pitiot
Checking Marie Sara
Checking Fabien Barthez
Checking Gil Bellows
Checking Félix Gray
Checking Kathy Bates
Checking Anny Duperey
Checking Christine Deviers-Joncour
Checking Jean-Paul Loth
Checking Mel Brooks
Checking Guy Lecluyse
Checking Claude Évin
Checking Fred Grandy
Checking Jacques Toubon
Checking Colette Castel
Checking Jacques Balutin
Checking Vahina Giocante
Checking Patrick Baud
Checking Florence Pernel
Checking Philippe Duquesne
Checking Rupert Grav

Checking Jean Nouvel
Checking Jean-Pierre Talbot
Checking George Hamilton
Checking Georges Kiejman
Checking Alex Goude
Checking Manu Joucla
Checking Kōji Kondō
Checking Romane Bohringer
Checking Emmanuelle Béart
Checking Lolita Lempicka
Checking Jean-Christophe Cambadélis
Checking Antonio Fargas
Checking Steve Martin
Checking Wim Wenders
Checking Roger Carel
Checking Jennifer Lawrence
Checking Frédéric Nihous
Checking Anne du Royaume-Uni
Checking Sylvie Vartan
Checking Laura Innes
Checking James Cameron
Checking Patrick Balkany
Checking Pierre Richard
Checking Hervé Morin
Checking Stephan Eicher
Checking Sean Penn
Checking Guilaine Chenu
Checking Nelson Piquet
Checking Guillermo Vilas
Checking Allain Bougrain-Dubourg
Checking Robert De Niro
Checking Michel Creton
Checking Pierre Douglas
Checking Jean-Jacques Sempé
Checking  Aphex Twin
Checking Alexandre Pesle
Checking Noam Kaniel
Checking Daniela Lumbroso
Checking Didier Auriol
Checking Madeleine Stowe
Checking Carole Bouquet
Checking 

Checking Alexis Trégarot
Checking Monica Bellucci
Checking Anthony Delon
Checking  Raël
Checking Diane Dufresne
Checking Jean-Louis Debré
Checking Angie Dickinson
Checking Jules-Édouard Moustic
Checking Jeane Manson
Checking Jean-Pierre Castaldi
Checking  Dani
Checking Jean-Luc Bideau
Checking Stella Stevens
Checking Julie Andrews
Checking Jimmy Carter
Checking Maxime Godart
Checking Pierre Mathieu
Checking Frank Delay
Checking Emma Colberti
Checking Alexandra Kazan
Checking Luis Fernandez
Checking Alicia Vikander
Checking Mickaël Vendetta
Checking Shannyn Sossamon
Checking Seann William Scott
Checking Neve Campbell
Checking Nathalie Vincent
Checking Clive Owen
Checking Tommy Lee
Checking Lou Bohringer
Checking Sara Forestier
Checking Christoph Waltz
Checking Tchéky Karyo
Checking Jesse Eisenberg
Checking Kate Winslet
Checking Sophie Dudemaine
Checking Sophie Favier
Checking Daniel Baldwin
Checking Caroline Loeb
Checking Karen Allen
Checking Brian Johnson
Checking Marie Laforêt
Checkin

Checking Patrick de Carolis
Checking Hervé Claude
Checking Jean-François Cayrey
Checking Sean Young
Checking Barbara Hendricks
Checking Richard Masur
Checking Philippe Ogouz
Checking Judith Magre
Checking Inés Sastre
Checking Cherry Jones
Checking Scarlett Johansson
Checking Isild Le Besco
Checking Mélanie Doutey
Checking Marjane Satrapi
Checking Anne Brochet
Checking Stéphane Freiss
Checking Basile de Koch
Checking Isabelle Bouysse
Checking Vincent Cassel
Checking Francis Cabrel
Checking Chantal Nobel
Checking Isabelle Renauld
Checking Thierry Samitier
Checking Alain Chabat
Checking Emir Kusturica
Checking Dwight Schultz
Checking Gaspard Ulliel
Checking Marcy Walker
Checking Karin Schubert
Checking Robert Marchand
Checking Michael Vartan
Checking Daniel Ducruet
Checking William Fichtner
Checking Philippe Delerm
Checking Jean-Pierre Dionnet
Checking Aimé Jacquet
Checking Aldo Maccione
Checking Mary Elizabeth Winstead
Checking John Galliano
Checking Ed Harris
Checking Jean-Charles de Ca

# Manual update

When there is missing celebrities or a mistake, we can fix manully some informations. For exemple there were several celebrities without the twitter account but I knew that they have one so we can add it

In [ ]:
def manual_update(name, twitter):
    conn = sqlite3.connect('F:/data/birthday.db')
    c = conn.cursor()
    c.execute("SELECT * FROM birthday WHERE name = ?", (name,))
    print(c.fetchone())
    c.execute("UPDATE birthday SET twitter = ? WHERE name = ?", (twitter, name))
    conn.commit()
    c.execute("SELECT * FROM birthday WHERE name = ?", (name,))
    print(c.fetchone())
    conn.close()

In [ ]:
# manual_update(" Cartouche", "@cartouche")

In [ ]:
# manual_update("Sandrine Quétier", "@SandQuetierOff")

In [ ]:
# manual_update(" Comte de Bouderbala", "@comtebouderbala")

In [ ]:
# manual_update(" Jul", "@jul")

In [ ]:
# manual_update(" Natoo", "@Nato_o")

In [ ]:
# manual_update("Laurent Baffie", "@lolobababa")

In [ ]:
# manual_update("Alain Bernard", "@alainbernard")

In [ ]:
# manual_update("Florence Parly", "@florence_parly")

In [ ]:
# manual_update("Christian Estrosi", "@cestrosi")

In [ ]:
# manual_update("Bernard Laporte", "@BernardLaporte_")

In [ ]:
# manual_update("Patrick Timsit", "@patricktimsit")

In [ ]:
# manual_update("Vincent Moscato", "@VINCENT_MOSCATO")

In [ ]:
# manual_update("Ève Angeli", "@eveangelioff")

In [ ]:
# add_data("1986-08-30", "Raphaël Carlier", "@Raphael_Carlier", "", "France", "0")

In [ ]:
# add_data("1986-07-07", "David Coscas", "@levraimcfly", "", "France", "0")

In [ ]:
# add_data("1992-08-07", "Damien Laguionie", "@terracid", "", "France", "0")

In [ ]:
# add_data("1992-01-08", "Thomas Iturralde", "@laink", "", "France", "0")

# Change birthday

Finally, there is maybe wrong date on the website. It was the case with Willy Rovelli. This section is done to update this

In [ ]:
name = "Willy Rovelli"
date = "1980-03-18"

In [ ]:
conn = sqlite3.connect('F:/data/birthday.db')
c = conn.cursor()
c.execute("SELECT * FROM birthday WHERE name = ?", (name,))
print(c.fetchone())
c.execute("UPDATE birthday SET birthday = ? WHERE name = ?", (date, name))
conn.commit()
c.execute("SELECT * FROM birthday WHERE name = ?", (name,))
print(c.fetchone())
conn.close()

# Check proposition

One improvement I've done is also host a website (simple one) to let people propose new celebrities. I'll try to review taht sometimes and this part is done for that purpose.

In [2]:
conn = sqlite3.connect('site/databases/proposition.db')
df = pd.read_sql_query("SELECT * FROM birthday", conn)
conn.close()
df.head()

,birthday,name,twitter,url,country,dead


In [ ]:
# conn = sqlite3.connect('site/databases/proposition.db')
# c = conn.cursor()
# c.execute('''DROP TABLE IF EXISTS birthday''')
# c.execute('''CREATE TABLE birthday
#              (birthday date_text, name text, twitter text, url text, country text, dead integer)''')
# conn.commit()
# conn.close()

# Conclusion

This Notebook is quite simple (and not very clean - a refactoring would be great...) but we saw here how to scrap data from a website and store it in a SQLite database. In the next Notebook, we will quickly explore the data we have.

Regarding the website, it's available @ http://nicolasmine.pythonanywhere.com/.